In [28]:
from dash import Dash, html, dcc, dash_table, Input, Output, State, callback
import pandas as pd
import plotly.express as px

app = Dash(__name__)

## pre-load the data
years = list(range(1970, 2024, 1))
df = pd.DataFrame()
for year in years:
    temp_df = pd.read_csv(f'../data/cleaned_fantasy_{year}.csv')
    temp_df['year'] = year
    df = pd.concat([df, temp_df])

default_table = pd.read_csv('../data/cleaned_fantasy_1970.csv')

# construct the lists for the dropdowns
positions = list(default_table['position'].unique())
positions.append('All')
options_dict = {'Top 100': 'rank', 'Mean':'mean', 'Median': 'median', 'Standard Deviation':'std'}
options_dict_reversed = {value: key for key, value in options_dict.items()}
sorting_variables_dict = {'Fantasy Points': 'fantasy_points', 'PPR Points': 'ppr_points', 
                     'Draft Kings Points': 'draft_kings_points', 'Fan Duel Points': 'fan_duel_points'}
sorting_variables_dict_reversed = {value: key for key, value in sorting_variables_dict.items()}

# set the layout
app.layout = [html.Title('Fantasy Football Dashboard'),
              html.H1('Fantasy Football Dashboard', style={'textAlign': 'center', 'color': 'white'}),
              html.Div(children=[
                  dcc.Dropdown(id='year-dropdown', options=[{'label': i, 'value': i} for i in range(1970, 2024, 1)], value=1970),
                  dcc.Dropdown(id='position-dropdown', options=[{'label': i, 'value': i} for i in positions], value='All'), 
                  dcc.Dropdown(id='columns', options=[{'label': i, 'value': i} for i in default_table.columns], 
                               value=['player', 'position', 'ppr_points', 'overall_rank'], multi=True)
              ]),
              html.Div(children=[]),
              html.Div(children=[
                  dash_table.DataTable(id='fantasy-football-table',data=default_table.to_dict('records'), 
                                       page_size=10, style_cell={'textAlign': 'center'})
              ]),
              dcc.Tabs(id='graph-tabs', value='tab-1', children=[
                dcc.Tab(label='Current Year', value='tab-1'),
                dcc.Tab(label='Trend', value='tab-2'),
              ]),
              html.Div(children=[
                    dcc.Dropdown(id='sorting-variable', options=[{'label': key, 'value': value} 
                                                                 for key, value in sorting_variables_dict.items()], value='ppr_points'),
                    dcc.Dropdown(id='options', options=[{'label': key, 'value': value} for key,value in options_dict.items()], value='rank')
                ]),
              html.Div(children=[
                dcc.Graph(id='fantasy-football-graph',figure={})])
            ]

@callback(
    Output('fantasy-football-graph', 'figure'),
    Input('graph-tabs', 'value'),
    Input('year-dropdown', 'value'),
    Input('sorting-variable', 'value'),
    Input('options', 'value')
)
def render_content(tab, year, sorting_variable, option):
    if tab == 'tab-1':
        return render_pie_chart(year, sorting_variable)
    elif tab == 'tab-2':
        return render_trend(sorting_variable, option)


@callback(
    Output('fantasy-football-table', 'data'),
    Input('position-dropdown', 'value'),
    Input('year-dropdown', 'value'),
    Input('columns', 'value')
)
def update_table(selected_position, selected_year, columns):
    df = pd.read_csv(f'../data/cleaned_fantasy_{selected_year}.csv')
    df = df.filter(columns).sort_values('ppr_points',ascending=False)
    if selected_position == 'All':
        return df.to_dict('records')
    else:
        return df[df['position'] == selected_position].to_dict('records')
    

def render_pie_chart(selected_year, sorting_variable):
    temp_df = df[df['year'] == selected_year]
    sorted_data = temp_df.sort_values(sorting_variable,ascending=False).head(100)
    sorting_variable_title = sorting_variables_dict_reversed[sorting_variable]
    color_dict = {'QB': 'blue', 'RB': 'green', 'WR': 'red', 'TE': 'purple'}
    fig = px.pie(sorted_data, sorted_data['position'], title = f'Top 100 Fantasy Football Players by Position in {selected_year} ({sorting_variable_title})', 
                 color='position', color_discrete_map=color_dict)
    return fig


def render_trend(sorting_variable='ppr_points', agg='rank'):

    # get top 100 by sorting_variable
    sorted_data = df.groupby('year').apply(lambda x: x.sort_values(sorting_variable,ascending=False), include_groups=False).reset_index()
    trend_data = sorted_data.groupby('year').head(100).reset_index(drop=True)
    sorting_variable_title = sorting_variables_dict_reversed[sorting_variable]

    if agg == 'rank':
        processed_df = trend_data.groupby(['year', 'position']).size().unstack().fillna(0)
        processed_df = processed_df.div(processed_df.sum(axis=1), axis=0) * 100
        processed_df = processed_df.round(2)
        fig = px.line(processed_df, x=processed_df.index, y=processed_df.columns, 
              title=f'Percentage of Top 100 Players by Position ({sorting_variable_title})', labels={'value': 'Percentage', 'year': 'Year'})
    else:
        # get agg, sorting_variable for chart title
        agg_title = options_dict_reversed[agg]
        
        # calculate the summary statistic of all players by year
        processed_df = trend_data.groupby(['year', 'position']).agg({sorting_variable: agg}).reset_index()
        processed_df = processed_df.pivot(index='year', columns='position', values=sorting_variable)

        # create moving average
        processed_df = processed_df.rolling(window=5).mean()
        fig = px.line(processed_df, x=processed_df.index, y=processed_df.columns, 
              title=f'Rolling {agg_title} (n=5) of {sorting_variable_title} for Top 100 Players', 
              labels={'value': f'Rolling {agg_title}', 'year': 'Year'})
    
    return fig

if __name__ == '__main__':
    app.run(debug=True)